### Problem with output v4.1

There is a problem, that the output from v4.1 has a bug, which is that the massdelta_cond is not 

In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import awkward as ak
from typing import Tuple

import matplotlib.pyplot as plt
from sdm_eurec4a.visulization import set_custom_rcParams, plot_one_one, label_from_attrs, save_figure
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from pySD.sdmout_src import supersdata
from pySD.sdmout_src import pygbxsdat, pysetuptxt, supersdata

default_colors = set_custom_rcParams()

from sdm_eurec4a import RepositoryPath  #

RP = RepositoryPath("levante")

In [ ]:
current_version = "v4.1"
cloud_id = 384

# microphysics = "null_microphysics"
microphysics = "condensation"
# microphysics = "collision_condensation"
# microphysics = "coalbure_condensation_large"
# microphysics = "coalbure_condensation_small"


versions = {
    "v4.0": dict(
        data_version="v4.0",
        cleo_version="v0.21.0",
        master_data_dir=Path(f"/home/m/m301096/CLEO/data/output_v4.0/{microphysics}"),
    ),
    "v4.1": dict(
        data_version="v4.1",
        cleo_version="v0.30.1",
        master_data_dir=Path(f"/home/m/m301096/CLEO/data/output_v4.1/{microphysics}"),
    ),
}

data_version: str = versions[current_version]["data_version"]
cleo_version: str = versions[current_version]["cleo_version"]

master_data_dir: Path = versions[current_version]["master_data_dir"]
fig_dir = RP.repo_dir / "results" / "mass_conservation" / data_version
fig_dir.mkdir(exist_ok=True, parents=True)

fig_title_appendix = (
    f"\nMicrop.: {microphysics} | Cloud: {cloud_id} | CLEO: {cleo_version} | Data: {data_version}"
)

# Load the datasets

data_dir = master_data_dir / f"cluster_{cloud_id}"
# data_dir = Path(f"/home/m/m301096/CLEO/data/output_v4.0/{microphysics}/cluster_81")
cloud_id = int(data_dir.name.split("_")[1])

# output_dir = data_dir / "processed"
# output_dir.mkdir(exist_ok=True, parents=False)

# output_path = output_dir / "eulerian_dataset.nc"
# output_path.parent.mkdir(exist_ok=True)

setupfile_path = data_dir / "config" / "eurec4a1d_setup.txt"
statsfile_path = data_dir / "config" / "eurec4a1d_stats.txt"
zarr_path = data_dir / "eurec4a1d_sol.zarr"
gridfile_path = data_dir / "share/eurec4a1d_ddimlessGBxboundaries.dat"


# read in constants and intial setup from setup .txt file
config = pysetuptxt.get_config(str(setupfile_path), nattrs=3, isprint=False)
consts = pysetuptxt.get_consts(str(setupfile_path), isprint=False)
gridbox_dict = pygbxsdat.get_gridboxes(str(gridfile_path), consts["COORD0"], isprint=False)

ds_zarr = xr.open_zarr(zarr_path, consolidated=False)
ds_zarr = ds_zarr.rename({"gbxindex": "gridbox"})
ds_zarr["time"] = np.round(ds_zarr["time"], 1)


ds_eulerian = xr.open_dataset(data_dir / "processed/eulerian_dataset.nc")
ds_eulerian["time"] = np.round(ds_eulerian["time"], 1)
# ds_eulerian['radius_bins'] = ds_eulerian['radius_bins'].where(ds_eulerian['radius_bins'] > 0, 1e-3)
ds_eulerian

Reading binary file:
 /home/m/m301096/CLEO/data/output_v4.1/condensation/cluster_384/share/eurec4a1d_ddimlessGBxboundaries.dat


In [ ]:
fig = plt.figure()
da = ds_eulerian["massdelta_condensation"].sel(time=slice(1500, 1510))
da.T.plot(cbar_kwargs=dict(label=label_from_attrs(da)))
fig.suptitle(f"{label_from_attrs(da, return_units = False)} given in zarr file" + fig_title_appendix)
fig.tight_layout()
save_figure(fig, fig_dir / "massdelta_cond_zarr.png")

## Attempt to understand conservation of mass in the setup

This is a bit like in a reservoir.

We have:
- $\mathit{I}$   Inflow of SDs 
- $\mathit{O}$   Outflow of SDs
- $\mathit{R}$   Reservoir of SDs within the domain 


For gridboxes $g \in \left[0, G\right]$
For time $t \in \left[0, T\right]$
For time $i \in \left[0, S\right]$


- The inflow at time t is given by the first time instance of a Droplet, in the top most SUBCLOUD gridbox. This should be the second timestep of existence.
- The outflow is given by the last timestep of existance.
- The resevoir are all OTHER values at this timestep for ALL gridboxes.

The differnce should be exactly in line with CLEOs output of ``masscondensed`` $dM$

$I = R + O + dM$

We need to use the represented mass for each SDs

In [ ]:
dataset = supersdata.SupersDataNew(
    dataset=ds_zarr,
    consts=consts,
)
dataset.set_attribute(dataset["sdId"].attribute_to_indexer_unique())
dataset.index_by_indexer(dataset["sdId"])
print(dataset)

---- Superdrop Properties -----
RHO_L = 998.203 Kg/m^3
RHO_SOL = 2016.5 Kg/m^3
MR_SOL = 0.05844277 Kg/mol
IONIC = 2.0
-------------------------------


Attribute coord1 not found in dataset
Attribute coord2 not found in dataset
Attributes:
--------------
sdId ()
coord: [0, 1, 2, 3, 4, 5, 6, ..., 1844218, 1844219, 1844220, 1844221, 1844222, 1844223]
1844224 * var * uint32
1844224 * var * int64
sdgbxindex ()
1844224 * var * uint32
xi ()
1844224 * var * float64
radius (micro-m)
1844224 * var * float64
msol (g)
1844224 * var * float64
coord3 (m)
1844224 * var * float64
time (s)
1844224 * var * float64
mass (kg)
1844224 * var * float64
mass_represented (kg)
1844224 * var * float64

Indexes:
--------------
sdId
[0, 1, 2, 3, 4, 5, 6, ..., 1844218, 1844219, 1844220, 1844221, 1844222, 1844223]



In [ ]:
id_slice = 3
time_slice = slice(0, 10)

timestep = np.arange(time_slice.start, time_slice.stop + 1)

time_data = dataset["time"].data[id_slice, time_slice].to_numpy()

lables = [f"t{j}= {i:.0f}s" for j, i in enumerate(time_data)]

fig, ax = plt.subplots(figsize=(3, 0.5))

ax: plt.Axes = ax

ax.scatter(time_data, time_data * 0, label="time")
for t in time_data:
    ax.annotate("", xy=(t + 2, 0), xytext=(t, 0), arrowprops=dict(arrowstyle="->"))
    ax.annotate("m", xy=(t, 1), fontsize=10, color=default_colors[0])

    # ax.annotate("", xy=(t + 2, 2), xytext=(t, 2),
    #         arrowprops=dict(arrowstyle="-", color = 'k'))

    # ax.annotate("e", xy=(t + 0.25, 2.5), fontsize=10, color = default_colors[1])
    # ax.annotate("e", xy=(t + 1.5, 2.5), fontsize=10, color = default_colors[2])


ax.set_yticks([])
ax.set_xticks(time_data, lables)
ax.set_ylim(-0.5, 3)
ax.set_xlim(-0.5, 6.2)

(-0.5, 6.2)

In [ ]:
def create_init_exit_reservoir_data(dataset: supersdata.SupersDataNew) -> Tuple[
    supersdata.SupersDataSimple,
    supersdata.SupersDataSimple,
    supersdata.SupersDataSimple,
]:

    data = dataset["time"].data
    mask = ak.num(data, axis=-1) > 1

    dataset_inflow = supersdata.SupersDataSimple([])
    dataset_outflow = supersdata.SupersDataSimple([])
    dataset_reservoir = supersdata.SupersDataSimple([])

    for key, attribute in dataset.attributes.items():
        data = attribute.data
        data = data[mask]
        inflow_array = data[:, 1]
        outflow_array = data[:, -1]
        reservoir_data = data[:, 1:-1]
        reservoir_data = ak.flatten(reservoir_data, axis=-1)

        dataset_inflow.set_attribute(
            supersdata.SupersAttribute(
                name=key, data=inflow_array, units=attribute.units, metadata=attribute.metadata
            )
        )
        dataset_outflow.set_attribute(
            supersdata.SupersAttribute(
                name=key, data=outflow_array, units=attribute.units, metadata=attribute.metadata
            )
        )
        dataset_reservoir.set_attribute(
            supersdata.SupersAttribute(
                name=key, data=reservoir_data, units=attribute.units, metadata=attribute.metadata
            )
        )

    return dataset_inflow, dataset_outflow, dataset_reservoir


dataset_inflow, dataset_outflow, dataset_reservoir = create_init_exit_reservoir_data(dataset)

In [ ]:
dataset_inflow.set_attribute(dataset_inflow["time"].attribute_to_indexer_unique())
dataset_inflow.index_by_indexer(dataset_inflow["time"])

dataset_outflow.set_attribute(dataset_outflow["time"].attribute_to_indexer_unique())
dataset_outflow.index_by_indexer(dataset_outflow["time"])

dataset_reservoir.set_attribute(dataset_reservoir["time"].attribute_to_indexer_unique())
dataset_reservoir.set_attribute(dataset_reservoir["sdgbxindex"].attribute_to_indexer_unique())
dataset_reservoir.index_by_indexer(dataset_reservoir["time"])
dataset_reservoir.index_by_indexer(dataset_reservoir["sdgbxindex"])

We can only use the timesteps up to the last time!!!!

In [ ]:
da_inflow = dataset_inflow.attribute_to_DataArray_reduction("mass_represented", reduction_func=ak.sum)
da_outflow = dataset_outflow.attribute_to_DataArray_reduction("mass_represented", reduction_func=ak.sum)
da_reservoir = dataset_reservoir.attribute_to_DataArray_reduction(
    "mass_represented", reduction_func=ak.sum
)

# outflow should be negative
da_outflow = -da_outflow
da_reservoir = da_reservoir

ds_box_model = xr.Dataset(
    {
        "inflow": da_inflow,
        "outflow": da_outflow,
        "reservoir": da_reservoir,
    }
)

# !!!!!!!!!!!!
# The data is now given in kg per timestep. We need to convert it to kg/s

ds_box_model = ds_box_model.rename({"sdgbxindex": "gridbox"})
ds_box_model = ds_box_model.fillna(0)
attrs = {key: ds_box_model[key].attrs.copy() for key in ds_box_model.data_vars}

ds_box_model = ds_box_model

ds_box_model["reservoir"] = ds_box_model["reservoir"].sum("gridbox")


for key in ds_box_model.data_vars:
    ds_box_model[key].attrs = attrs[key]

ds_box_model["inflow"].attrs["units"] = "kg dT^{-1}"
ds_box_model["outflow"].attrs["units"] = "kg dT^{-1}"
ds_box_model["reservoir"].attrs["units"] = "kg"

ds_box_model["reservoir"][0] = ds_box_model["inflow"][0]


ds_box_model["inflow_integrate"] = ds_box_model["inflow"].cumsum("time", keep_attrs=True)
ds_box_model["inflow_integrate"] = ds_box_model["inflow_integrate"].shift(time=0)
ds_box_model["inflow_integrate"].attrs["units"] = "kg"

ds_box_model["outflow_integrate"] = ds_box_model["outflow"].cumsum("time", keep_attrs=True)
ds_box_model["outflow_integrate"] = ds_box_model["outflow_integrate"].shift(time=0)
ds_box_model["outflow_integrate"].attrs["units"] = "kg"

ds_box_model["reservoir_change"] = ds_box_model["reservoir"].diff("time")
ds_box_model["reservoir_change"] = ds_box_model["reservoir_change"].shift(time=0)
ds_box_model["reservoir_change"].attrs["units"] = "kg dT^{-1}"

ds_box_model["reservoir_change"][0] = ds_box_model["reservoir"][0] - 0


# add the source terms

da_source = ds_eulerian["massdelta_condensation"]
# make sure to have kg per gridbox
da_source = da_source * ds_eulerian["gridbox_volume"]
# make sure to have per timestep NOT per second
da_source = da_source * ds_eulerian["time"].diff("time").mean()
# only use sub cloud layer gridboxes
da_source = da_source.sel(gridbox=slice(0, ds_eulerian["gridbox"].max() - 1))


ds_box_model["source"] = da_source.isel(gridbox=slice(None, None)).sum("gridbox").shift(time=0)
ds_box_model["source"].attrs = dict(
    long_name="Source term",
    units="kg dT^{-1}",
)

ds_box_model["source_integrate"] = ds_box_model["source"].cumsum("time", keep_attrs=True)
ds_box_model["source_integrate"].attrs = dict(
    long_name="Integrated source term",
    units="kg",
)

ds_box_model["sum"] = (
    -ds_box_model["reservoir_change"]
    + ds_box_model["inflow"]
    + ds_box_model["outflow"]
    + ds_box_model["source"]
)
ds_box_model["sum_integrate"] = (
    -ds_box_model["reservoir"]
    + ds_box_model["inflow_integrate"]
    + ds_box_model["outflow_integrate"]
    + ds_box_model["source_integrate"]
)

ds_box_model = ds_box_model.sel(time=slice(0, 3000)).fillna(0)

In [ ]:
ds_box_model_rolling = ds_box_model.rolling(time=2, center=True).mean()

### Scatter 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6), sharex=True)

x = ds_box_model["time"]

for var, marker in zip(["reservoir_change", "inflow", "outflow", "source"], ["o", "^", "v", "P"]):

    da = ds_box_model[var]
    ax.scatter(
        x,
        da,
        label=var,
        marker=marker,
        alpha=0.7,
        s=5,
    )

ax.scatter(
    x,
    ds_box_model["sum"],
    label="Sum over all terms",
    alpha=0.7,
    color="k",
    s=5,
    marker=".",
)
# (ds_box_model['inflow_integrate'] + ds_box_model['reservoir'] + ds_box_model['outflow_integrate']+ ds_box_model['source_integrate']).diff('time').plot(label = 'SUM reconstr', marker = '.', alpha = 0.7, markersize = 5, linestyle = 'None', ax = axs[1])
ax.legend(loc="center right", facecolor="w")
# plt.axhline(0.00015, color = 'k', linestyle = '--')
# plt.axhline(-0.00015, color = 'k', linestyle = '--')
ax.set_yscale("symlog", linthresh=1e-8)

fig.suptitle("Mass conservation box model for instantanious " + fig_title_appendix)
fig.tight_layout()
save_figure(fig, fig_dir / "mass_conservation_box_model_instantanious", formats=[".png"])

/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/visulization.py:1051: UserWarning: Got formats in tuple or list format. Converting to dict.
  warnings.warn("Got formats in tuple or list format. Converting to dict.")


In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

x = ds_box_model["time"]

_ax = axs[0]
for var in ["reservoir_change", "inflow", "outflow", "source"]:

    da = ds_box_model[var]
    axs[0].plot(
        x,
        da,
        label=var,
        alpha=0.7,
    )

axs[0].plot(x, ds_box_model["sum"], label="Sum over all terms", alpha=0.7, color="k")

for var in ["reservoir", "inflow_integrate", "outflow_integrate", "source_integrate"]:

    da = ds_box_model[var]
    axs[1].plot(
        x,
        da,
        label=var,
        alpha=0.7,
    )

axs[1].plot(x, ds_box_model["sum_integrate"], label="Sum over all terms", alpha=0.7, color="k")


axs[0].set_ylabel("Terms " + label_from_attrs(ds_box_model["source"], return_name=False))
axs[0].set_title("Instantanious terms in the CLEO box model")
axs[1].set_ylabel(
    "Integrated terms " + label_from_attrs(ds_box_model["source_integrate"], return_name=False)
)
axs[1].set_title("Integrated terms in the CLEO box model")

for _ax in axs:
    # _ax.set_yscale("symlog", linthresh=1e-10)
    _ax.set_xlabel(label_from_attrs(ds_box_model["time"]))
    _ax.legend(loc="center right", facecolor="w", framealpha=1)

fig.suptitle(
    "Evolution of terms in the CLEO box model for instantanious (top) and integrated version (bottom) "
    + fig_title_appendix
)
fig.tight_layout()
save_figure(fig, fig_dir / "mass_conservation_box_model_comparison", formats=[".png"])

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

x = ds_box_model["time"]

_ax = axs[0]
for var, marker in zip(["reservoir_change", "inflow", "outflow", "source"], ["o", "^", "v", "P"]):

    da = ds_box_model[var]
    axs[0].scatter(
        x,
        da,
        label=var,
        marker=marker,
        alpha=0.7,
        s=5,
    )

axs[0].scatter(x, ds_box_model["sum"], label="Sum over all terms", alpha=0.7, color="k", s=5, marker=".")

for var, marker in zip(
    ["reservoir", "inflow_integrate", "outflow_integrate", "source_integrate"], ["o", "^", "v", "P"]
):

    da = ds_box_model[var]
    axs[1].scatter(
        x,
        da,
        label=var,
        marker=marker,
        alpha=0.7,
        s=5,
    )

axs[1].scatter(
    x,
    ds_box_model["sum_integrate"],
    label="Sum over all terms",
    alpha=0.7,
    color="k",
    s=5,
    marker=".",
)


axs[0].set_ylabel("Terms " + label_from_attrs(ds_box_model["source"], return_name=False))
axs[0].set_title("Instantanious terms in the CLEO box model")
axs[1].set_ylabel(
    "Integrated terms " + label_from_attrs(ds_box_model["source_integrate"], return_name=False)
)
axs[1].set_title("Integrated terms in the CLEO box model")

for _ax in axs:
    _ax.set_yscale("symlog", linthresh=1e-8)
    _ax.set_xlabel(label_from_attrs(ds_box_model["time"]))
    _ax.legend(loc="center right", facecolor="w", framealpha=1)

fig.suptitle(
    "Evolution of terms in the CLEO box model for instantanious (top) and integrated version (bottom) "
    + fig_title_appendix
)
fig.tight_layout()
save_figure(fig, fig_dir / "mass_conservation_box_model_comparison_symlog", formats=[".png"])

### Scatter of error term

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 6))

x = ds_box_model["time"]
axs[0].plot(
    x,
    ds_box_model["sum"],
    label=r"$- \frac{dR}{dt} + I + O + S$",
    marker=".",
    alpha=0.7,
    markersize=5,
    linestyle="None",
)

axs[1].plot(
    x,
    ds_box_model["sum_integrate"],
    label=r"$-R + \int_{0}^{t}\left(I + O + S \right) dt$",
    marker=".",
    alpha=0.7,
    markersize=5,
    linestyle="None",
)


axs[0].set_ylabel("Terms " + label_from_attrs(ds_box_model["source"], return_name=False))
axs[0].set_title("Instantanious terms in the CLEO box model")
axs[1].set_ylabel(
    "Integrated terms " + label_from_attrs(ds_box_model["source_integrate"], return_name=False)
)
axs[1].set_title("Integrated terms in the CLEO box model")

for _ax in axs:
    _ax.set_xlabel(label_from_attrs(ds_box_model["time"]))
    _ax.legend()
    # _ax.set_xlim(1000, 1100)

fig.suptitle(
    "Mass conservation box model for instantanious (top) and integrated version (bottom) "
    + fig_title_appendix
)
fig.tight_layout()
save_figure(fig, fig_dir / "mass_conservation_box_model_comparison", formats=[".png"])

### Step by step comparison

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8), sharex=True)

axs: Tuple[plt.Axes, plt.Axes, plt.Axes, plt.Axes] = axs.flatten()

axs[0].plot(ds_box_model["time"], ds_box_model["inflow"], label="I (inflow mass in topmost gridbox)")
axs[0].plot(ds_box_model["time"], ds_box_model["outflow"], label="O (outflow mass in lowest gridbox)")

axs[0].legend(loc="center right")

axs[1].plot(ds_box_model["time"], ds_box_model["inflow"] + ds_box_model["outflow"], label="$I + O$")

axs[1].legend(loc="center right")

data = ds_box_model["reservoir_change"]

axs[1].plot(data["time"], data, label=r"$\frac{dR}{dt}$ reservoir")
axs[1].legend(loc="upper right")

data = ds_box_model["inflow"] + ds_box_model["outflow"] - ds_box_model["reservoir_change"]
axs[2].plot(data["time"], data, label=r"$-\frac{dR}{dt} + I + O$")
axs[2].plot(
    ds_box_model["source"]["time"],
    ds_box_model["source"],
    label="S",
)

axs[2].legend(loc="upper left")

data = (
    ds_box_model["inflow"]
    + ds_box_model["outflow"]
    - ds_box_model["reservoir_change"]
    + ds_box_model["source"]
)
axs[3].plot(data["time"], data, label=r"$-\frac{dR}{dt} + I + O + S$")
axs[3].legend(loc="upper left")

for _ax in axs:
    _ax.set_ylabel("Terms " + label_from_attrs(ds_box_model["source"], return_name=False))
    _ax.set_xlabel(label_from_attrs(ds_box_model["time"]))

fig.suptitle("Step by step comparison of terms in CLEO box model" + fig_title_appendix)
fig.tight_layout()
save_figure(fig, fig_dir / "step_by_step_conservation", formats=[".png"])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8), sharex=True)

axs: Tuple[plt.Axes, plt.Axes, plt.Axes, plt.Axes] = axs.flatten()

axs[0].plot(
    ds_box_model["time"], ds_box_model["inflow_integrate"], label="I (inflow mass in topmost gridbox)"
)
axs[0].plot(
    ds_box_model["time"], ds_box_model["outflow_integrate"], label="O (outflow mass in lowest gridbox)"
)

axs[0].legend(loc="center right")

axs[1].plot(
    ds_box_model["time"],
    ds_box_model["inflow_integrate"] + ds_box_model["outflow_integrate"],
    label="$I - O$",
)

axs[1].legend(loc="center right")

data = ds_box_model["reservoir"]

axs[1].plot(data["time"], data, label="R reservoir")
axs[1].legend(loc="upper right")

data = ds_box_model["inflow_integrate"] + ds_box_model["outflow_integrate"] - ds_box_model["reservoir"]
axs[2].plot(data["time"], data, label=r"$I-O-\frac{dR}{dt}$")
axs[2].plot(
    ds_box_model["source_integrate"]["time"],
    ds_box_model["source_integrate"],
    label="E",
)

axs[2].legend(loc="upper left")

data = (
    ds_box_model["inflow_integrate"]
    + ds_box_model["outflow_integrate"]
    - ds_box_model["reservoir"]
    + ds_box_model["source_integrate"]
)
axs[3].plot(data["time"], data, label=r"$I-O-\frac{dR}{dt} - S$")
axs[3].legend(loc="upper left")

for _ax in axs:
    _ax.set_ylabel(
        "Integrated terms " + label_from_attrs(ds_box_model["source_integrate"], return_name=False)
    )
    _ax.set_xlabel(label_from_attrs(ds_box_model["time"]))

fig.suptitle("Step by step comparison of integrated terms in CLEO box model" + fig_title_appendix)
fig.tight_layout()
save_figure(fig, fig_dir / "step_by_step_conservation_integrated", formats=[".png"])

### Mass loss due to wrong storing of data

In [ ]:
data = ds_box_model["inflow"] + ds_box_model["outflow"] - ds_box_model["reservoir_change"]
data = data

y_init = data.sel(time=slice(1500, None))
y_init.attrs.update(
    long_name=r"Sum of other terms $\left(I-O-\frac{dR}{dt}\right)$",
    units=ds_box_model["inflow"].attrs["units"],
)

x_init = -ds_box_model["source"].sel(time=slice(1500, None))
x_init.attrs.update(
    long_name=r"Source term (evaporation) $(S)$",
)

y, yerr = y_init.mean("time"), y_init.std("time")
x, xerr = x_init.mean("time"), x_init.std("time")


mini = min(x_init.min(), y_init.min())
maxi = max(x_init.max(), y_init.max())

bins = np.linspace(mini, maxi, 20)

fig, ax = plt.subplots(1, 1, figsize=(8, 7))
_, _, _, qm = ax.hist2d(
    x_init,
    y_init,
    bins=bins,
    cmap="Blues",
)


fig.colorbar(qm, ax=ax, label="Number of timesteps")
ax.errorbar(
    x=x,
    y=y,
    xerr=xerr,
    yerr=yerr,
    fmt=".",
    color="k",
    markersize=10,
    zorder=10,
    label="Mean and std. error of mean",
)
ax.axhline(y, color="grey", linestyle=":")
ax.axvline(x, color="grey", linestyle=":")

plot_one_one(ax, linestyle="--", color="grey")

x_label = label_from_attrs(x_init) + f"\n{x.values:.3e} +- {xerr.values:.3e}"
y_label = label_from_attrs(y_init) + f"\n{y.values:.3e} +- {yerr.values:.3e}"

ax.set_xlabel(x_label)
ax.set_ylabel(y_label)

fig.suptitle(
    "Comparison of source term and sum of other terms in CLEO box model for stationary state"
    + fig_title_appendix
)
fig.tight_layout()
save_figure(fig, fig_dir / "source_vs_sum_of_other_terms", formats=[".png"])

/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/visulization.py:1051: UserWarning: Got formats in tuple or list format. Converting to dict.
  warnings.warn("Got formats in tuple or list format. Converting to dict.")


In [ ]:
data = ds_box_model["inflow"] + ds_box_model["outflow"] - ds_box_model["reservoir_change"]
data = data

y_init = data.sel(time=slice(1500, None))
y_init.attrs.update(
    long_name=r"Sum of other terms $\left(I-O-\frac{dR}{dt}\right)$",
    units=ds_box_model["inflow"].attrs["units"],
)

x_init = -ds_box_model_rolling["source"].sel(time=slice(1500, None))
x_init.attrs.update(
    long_name=r"Source term (evaporation) $(S)$",
)

y, yerr = y_init.mean("time"), y_init.std("time")
x, xerr = x_init.mean("time"), x_init.std("time")


mini = min(x_init.min(), y_init.min())
maxi = max(x_init.max(), y_init.max())

bins = np.linspace(mini, maxi, 20)

fig, ax = plt.subplots(1, 1, figsize=(8, 7))
_, _, _, qm = ax.hist2d(
    x_init,
    y_init,
    bins=bins,
    cmap="Blues",
)


fig.colorbar(qm, ax=ax, label="Number of timesteps")
ax.errorbar(
    x=x,
    y=y,
    xerr=xerr,
    yerr=yerr,
    fmt=".",
    color="k",
    markersize=10,
    zorder=10,
    label="Mean and std. error of mean",
)
ax.axhline(y, color="grey", linestyle=":")
ax.axvline(x, color="grey", linestyle=":")

plot_one_one(ax, linestyle="--", color="grey")

x_label = label_from_attrs(x_init) + f"\n{x.values:.3e} +- {xerr.values:.3e}"
y_label = label_from_attrs(y_init) + f"\n{y.values:.3e} +- {yerr.values:.3e}"

ax.set_xlabel(x_label)
ax.set_ylabel(y_label)

fig.suptitle(
    "Comparison of source term and sum of other terms in CLEO box model for stationary state"
    + fig_title_appendix
)
fig.tight_layout()
save_figure(fig, fig_dir / "source_vs_sum_of_other_terms-rolling_mean", formats=[".png"])

/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/visulization.py:1051: UserWarning: Got formats in tuple or list format. Converting to dict.
  warnings.warn("Got formats in tuple or list format. Converting to dict.")


### Compare for all clouds

In [ ]:
ds_all_clouds = xr.open_dataset(master_data_dir / "combined" / "conservation_dataset_combined.nc")
ds_all_clouds["source_rolling"] = (
    ds_all_clouds["source"].rolling(time=2, center=True).mean(keep_attrs=True)
)
ds_all_clouds = ds_all_clouds.sel(time=slice(1500, 3590))
ds_all_clouds

In [ ]:
x_data = ds_all_clouds["inflow"] + ds_all_clouds["outflow"] - ds_all_clouds["reservoir_change"]
x_data.attrs.update(
    long_name=r"Sum of other terms $\left(I-O-\frac{dR}{dt}\right)$",
    units=ds_box_model["inflow"].attrs["units"],
)


y_data = -ds_all_clouds["source"]
y_rolling_data = -ds_all_clouds["source_rolling"]

corr = xr.corr(x_data.mean("time"), y_data.mean("time"), dim="cloud_id")
corr_rolling = xr.corr(x_data.mean("time"), y_rolling_data.mean("time"), dim="cloud_id")

plt.figure()
plt.errorbar(
    x=x_data.mean("time"),
    y=y_data.mean("time"),
    xerr=x_data.std("time"),
    yerr=y_data.std("time"),
    fmt=".",
    label=f"Source term. R = {corr.values:.2f}",
)

plt.errorbar(
    x=x_data.mean("time"),
    y=y_rolling_data.mean("time"),
    xerr=x_data.std("time"),
    yerr=y_rolling_data.std("time"),
    fmt=".",
    label=f"Source term rolling mean. R = {corr_rolling.values:.2f}",
)
plot_one_one(plt.gca(), linestyle="--", color="grey")

plt.legend()
plt.xlabel(label_from_attrs(x_data))
plt.ylabel(label_from_attrs(y_data))

plt.figure()

xm = x_data.mean("time")
ym = y_data.mean("time")
xerr = x_data.std("time")
yerr = y_data.std("time")
sigma = xr.corr(x_data, y_data, dim="time")


relative = 100 * ((xm / ym - 1) ** 2) ** (0.5)
relative.attrs.update(long_name="Relative error between source term and sum of other terms", units=r"\%")
# relative_err = np.sqrt((xerr / xm)**2 + (yerr / ym)**2 - 2 * sigma / (xm * ym))
relative_err = 0

plt.errorbar(
    x=x_data.mean("time"),
    xerr=x_data.std("time"),
    y=relative,
    yerr=relative_err,
    fmt=".",
    label=f"Source term rolling mean. R = {corr_rolling.values:.2f}",
)
# plot_one_one(plt.gca(), linestyle = '--', color = 'grey')
plt.xscale("log")
# plt.yscale('symlog', linthresh = 1e-8)
plt.legend()
plt.xlabel(label_from_attrs(x_data))
plt.ylabel(label_from_attrs(relative, name_width=20))

Text(0, 0.5, 'Relative error\nbetween source term\nand sum of other\nterms $\\left[  \\%  \\right]$')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

x = ds_box_model_rolling["time"]

_ax = axs[0]
for var, marker in zip(["reservoir_change", "inflow", "outflow", "source"], ["o", "^", "v", "P"]):

    da = ds_box_model_rolling[var]
    axs[0].scatter(
        x,
        da,
        label=var,
        marker=marker,
        alpha=0.7,
        s=5,
    )

axs[0].scatter(
    x, ds_box_model_rolling["sum"], label="Sum over all terms", alpha=0.7, color="k", s=5, marker="."
)

for var, marker in zip(
    ["reservoir", "inflow_integrate", "outflow_integrate", "source_integrate"], ["o", "^", "v", "P"]
):

    da = ds_box_model_rolling[var]
    axs[1].scatter(
        x,
        da,
        label=var,
        marker=marker,
        alpha=0.7,
        s=5,
    )

axs[1].scatter(
    x,
    ds_box_model_rolling["sum_integrate"],
    label="Sum over all terms",
    alpha=0.7,
    color="k",
    s=5,
    marker=".",
)


axs[0].set_ylabel("Terms " + label_from_attrs(ds_box_model_rolling["source"], return_name=False))
axs[0].set_title("Instantanious terms in the CLEO box model")
axs[1].set_ylabel(
    "Integrated terms " + label_from_attrs(ds_box_model_rolling["source_integrate"], return_name=False)
)
axs[1].set_title("Integrated terms in the CLEO box model")

for _ax in axs:
    _ax.set_yscale("symlog", linthresh=1e-4)
    _ax.set_xlabel(label_from_attrs(ds_box_model_rolling["time"]))
    _ax.legend(loc="center right", facecolor="w", framealpha=1)

fig.suptitle(
    "Evolution of terms in the CLEO box model for instantanious (top) and integrated version (bottom) "
    + fig_title_appendix
)
fig.tight_layout()
save_figure(fig, fig_dir / "rolling_mass_conservation_box_model_comparison_symlog", formats=[".png"])

In [ ]:
fig, axs = plt.subplots(ncols=2, sharex=True)
ax = axs[0]
ax_t = ax.twinx()
ax_int = axs[1]
ax_int_t = ax_int.twinx()

x_init = -ds_box_model["source"].sel(time=slice(1500, None))
x_init2 = x_init.rolling(time=2, center=True).mean()


x_init.plot(ax=ax, marker="x", linestyle="None", label="org")
x_init2.plot(ax=ax, marker="+", linestyle="None", label="shifted")
(x_init / x_init2).plot(ax=ax_t, marker=".", linestyle="None", color="k")
print(x_init.mean("time") / x_init2.mean("time"))
print(x_init.mean("time") - x_init2.mean("time"))

ax.legend()
(x_init.cumsum("time")).plot(ax=ax_int, marker=".", linestyle="None")
(x_init2.cumsum("time")).plot(ax=ax_int, marker=".", linestyle="None")
(x_init.cumsum() / x_init2.cumsum()).plot(ax=ax_int_t, marker=".", linestyle="None", color="k")

print(x_init.sum("time") / x_init2.sum("time"))
print(x_init.sum("time") - x_init2.sum("time"))

# ax.set_xlim(1500, 1540)
fig.tight_layout()

<xarray.DataArray 'source' ()> Size: 8B
array(1.00134041)
<xarray.DataArray 'source' ()> Size: 8B
array(0.00410313)
<xarray.DataArray 'source' ()> Size: 8B
array(1.00267553)
<xarray.DataArray 'source' ()> Size: 8B
array(6.14253706)


In [ ]:
A_init = -100 * (ds_box_model["outflow"]).sel(time=slice(1500, None))
B_init = ds_box_model["inflow"].sel(time=slice(1500, None))

A, sigma_A = mean_and_stderror_of_mean(A_init, dims="time")
B, sigma_B = mean_and_stderror_of_mean(B_init, dims="time")

sigma_AB = xr.cov(A_init, B_init, dim="time")

f = A / B
sigma_f = np.abs(f) * np.sqrt(
    (sigma_A / A) ** 2
    + (sigma_B / B) ** 2
    # - 2 * (sigma_AB / (A * B)) ** 2
)

data = A_init / B_init

fig, axs = plt.subplots(ncols=2, figsize=(10, 4), sharey=True, width_ratios=[3, 1])

axs: Tuple[plt.Axes, plt.Axes] = axs


axs[0].plot(data["time"], data, alpha=1, color="grey", zorder=-1)
# axs[0].axhline(100, color="k", linestyle="--")
axs[0].axhline(f, color="r", linestyle="-", label="mean")
axs[0].fill_between(
    data["time"],
    f - sigma_f,
    f + sigma_f,
    color="r",
    alpha=0.3,
    label="f $\\pm$ sigma_f" + f"\n{f:.2f} $\\pm$ {sigma_f:.2f} %",
)
m, s = data.mean("time"), data.std("time")
axs[0].fill_between(
    data["time"],
    m - s,
    m + s,
    color="b",
    alpha=0.1,
    label="mean $\\pm$ STD" + f"\n{m:.2f} $\\pm$ {s:.2f} %",
)
axs[0].set_xlim(0, 3500)
axs[0].legend()
axs[0].set_ylabel("Outflow / Inflow [%]")
axs[0].set_xlabel("Time [s]")

p, b, _ = axs[1].hist(
    data.sel(time=slice(0, None)),
    # bins=np.arange(0, 300, 10),
    density=True,
    alpha=0.5,
    color="grey",
    orientation="horizontal",
)
# axs[1].axhline(100, color="k", linestyle="--")

axs[1].fill_between(
    [0, 1.1 * np.max(p)],
    f - sigma_f,
    f + sigma_f,
    color="r",
    alpha=0.3,
    label="f $\\pm$ sigma_f" + f"\n{f:.2f} $\\pm$ {sigma_f:.2f} %",
)

axs[1].fill_between(
    [0, 1.1 * np.max(p)],
    m - s,
    m + s,
    color="b",
    alpha=0.1,
    label="mean $\\pm$ STD" + f"\n{m:.2f} $\\pm$ {s:.2f} %",
)

axs[1].set_xlabel("Density")
axs[1].legend()

for _ax in axs:
    _ax.axhline(m, color="b", linestyle="-", label="mean")
    _ax.axhline(f, color="r", linestyle="-", label="mean")

fig.suptitle("Outflow fraction (Outflow / Inflow)")
fig.tight_layout()
fig.savefig(fig_dir / "outflow_fraction.png", dpi=300)